In [1]:
# imports
import plotly.graph_objects as go
import pandas as pd
import os
import numpy as np 
import json
import plotly.io as pio
pio.renderers.default = "browser"

C:\Users\marie\AppData\Local\Temp\ipykernel_10332\3435604520.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# data sources
pc_path = r"C:\Users\marie\rep_codes\udder_project\udder_processing\point_clouds"
feature_path = r"C:\Users\marie\rep_codes\udder_project\udder_processing\features_dict"
udder_pc_path = os.path.join(pc_path, "udder")
quarter_pc_path = os.path.join(pc_path, "quarters")
keypoint_pc_path = os.path.join(pc_path, "keypoints")
# teat_pc_path = os.path.join(pc_path, "keypoints")
teat_pc_path = os.path.join(pc_path, "teat")
teat_len_path = os.path.join(feature_path,  "teat_length")
distance_path = os.path.join(feature_path,  "distance")
filenames = [file.replace(".json", "") for file in os.listdir(teat_len_path)]

color_dict = {'lf': 'cyan', 'rf': 'skyblue', 'lb': 'royalblue', 'rb': 'dodgerblue', 'front':'cyan', 
             'right': 'skyblue', 'left': 'royalblue', 'back': 'dodgerblue', 'udder': 'plum'}

file_dict = {}
for file in filenames:
    cow = file.split("_")[0]
    frame = file.split("_")[-1]
    if cow in set(file_dict.keys()):
        file_dict[cow][frame] = file
    else:
        file_dict[cow] ={frame: file}

cow_list = np.unique(list(file_dict.keys()))
df = pd.read_csv(os.path.join("data", "feature_table.csv"))

C:\Users\marie\AppData\Local\Temp\ipykernel_10332\617256814.py:26: DtypeWarning:

Columns (21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
keyword_dict = [{'label':'volume', 'value':'vol'},
                {'label':'surface area', 'value': 'sarea'},
                {'label':'circularity', 'value':'circ'},
                {'label':'excentricity', 'value': 'exc'},
                {'label': 'Euclidean distance', 'value': 'eu'},
                {'label': 'geodesic distance', 'value': 'gd'},
                {'label': 'teat length', 'value': 'len'}]
statvar_dict = [{'label': 'mean', 'value': 'mean'},{'label': 'median', 'value': 'median'},]

In [4]:
def udder_plot(file):
    udder_pc = np.load(os.path.join(udder_pc_path, file +".npy"))

    with open(os.path.join(quarter_pc_path, file + ".json")) as f:
        quarter_dict = json.load(f)
    
    with open(os.path.join(keypoint_pc_path, file + ".json")) as f:
        keypoint_dict = json.load(f)
    
    with open(os.path.join(distance_path, file + ".json")) as f:
        distance_dict = json.load(f)
    
    with open(os.path.join(teat_pc_path, file + ".json")) as f:
        teat_pc_dict = json.load(f)
    
    with open(os.path.join(teat_len_path, file + ".json")) as f:
        teat_len_dict = json.load(f)
    
    kploc = np.zeros((5,3)) # use five so the Euclidean distance lines close
    for i, teat in enumerate(["lf", "rf", "rb", "lb", "lf"]):
        kploc[i, :] = keypoint_dict[teat]["xyz_tf"]
    
    bottoms = np.zeros((4,3))
    tips = np.zeros((4,3))
    lines = {}
    for i, key in enumerate(teat_len_dict.keys()):
        bottoms[i, :] = teat_len_dict[key]["bottom"]
        tips[i, :] = teat_len_dict[key]["tip"]
        lines[key] = np.row_stack([teat_len_dict[key]["tip"], teat_len_dict[key]["bottom"]])
    
    points = udder_pc
    fig =  go.Figure(data=[go.Scatter3d(x = points[:, 0], y = points[:, 1], z=points[:, 2],mode='markers',
     marker=dict(size=2, color=points[:, 2], colorscale='gray'), name = "Udder")])

    fig.add_trace(go.Scatter3d(x = points[:, 0], y = points[:, 1], z=points[:, 2],mode='markers', marker=dict(size=2, color="steelblue" , opacity=0.3), name = "Udder2"))
    for i, key in enumerate(quarter_dict):
        sl = True if i == 0 else False
        points = np.array(quarter_dict[key])
        c = color_dict[key]
        fig.add_trace(go.Scatter3d(x= points[:, 0], y = points[:, 1], z=points[:, 2], mode='markers', marker=dict(color=c, size = 2), name = "quarters", legendgroup = "quarters", showlegend = sl))
    
    for i, key in enumerate(distance_dict):
        sl = True if i == 0 else False
        points = np.array(distance_dict[key]['path'])
        fig.add_trace(go.Scatter3d(x= points[:, 0], y = points[:, 1], z=points[:, 2], mode='markers', marker=dict(color='red', size = 2), name = "geodesic", legendgroup= "geodesic", showlegend = sl))
    
    for i, key in enumerate(teat_pc_dict.keys()):
        sl = True if i == 0 else False
        name = 'obs_pts'
        points = np.array(teat_pc_dict[key][name])
        fig.add_trace(go.Scatter3d(x= points[:, 0], y = points[:, 1], z=points[:, 2], mode='markers', marker=dict(color="steelblue", size = 2), name = name, legendgroup= name, showlegend = sl))
        name = 'pred_pts'
        points = np.array(teat_pc_dict[key][name])
        fig.add_trace(go.Scatter3d(x= points[:, 0], y = points[:, 1], z=points[:, 2], mode='markers', marker=dict(color="gray", size = 2), name = name, legendgroup= name, showlegend = sl))
        
    for i, key in enumerate(lines.keys()):
        sl = True if i == 0 else False
        data = lines[key]
        fig.add_trace(go.Scatter3d(x = data[:, 0], y = data[:, 1], z= data[:, 2], mode='lines', line=dict(color="red",  width = 3), name = "teat_len", legendgroup="teat_len", showlegend = sl))
    
    fig.add_trace(go.Scatter3d(x = tips[:, 0], y = tips[:, 1], z= tips[:, 2], mode='markers', marker=dict(color="red", size = 4), name = "teat_point1", legendgroup ="teat_point1", showlegend = False))
    fig.add_trace(go.Scatter3d(x = bottoms[:, 0], y = bottoms[:, 1], z= bottoms[:, 2], mode='markers', marker=dict(color="red", size = 4), name = "teat_point2", legendgroup ="teat_point2"))
        
    points = kploc
    fig.add_trace(go.Scatter3d(x= points[:, 0], y = points[:, 1], z=points[:, 2], mode='lines', line=dict(color="red", width = 2), name = "euclidean"))
    
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
    fig.update_layout(autosize=False, width=1000, height=800)
    return fig 

In [5]:

fig = udder_plot(file)

file = file_dict['1240']['222']
dirpath = os.getcwd()
plotdir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), "adsa", "examples")
fig.write_html(os.path.join(plotdir, "teat_example.html"))